### Notebook to go through all mat files and get fully hand labeled trials

In [1]:
from pathlib import Path
from tqdm import tqdm
import shutil
import os
from scipy.io import loadmat
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm
import pandas as pd

In [2]:
columns = ["mat_path", "full_hand_labels"]
df = pd.DataFrame(columns=columns)

In [3]:
df

,mat_path,full_hand_labels


In [4]:
mat_dir = Path("/data/caitlin/mat_files/")

In [5]:
def check_all_behaviors(ethogram: np.ndarray):
    for row in ethogram:
        if not row.any():
            return False
    return True

In [6]:
mat_paths = sorted(mat_dir.glob('*'))
len(mat_paths)

696

In [7]:
mat_errors = list()
def get_hand_labels(mat_path): 
    hand_labels = dict()

    m = loadmat(mat_path)
    try:
        num_trials = m["data"]["trial"].shape[0]
        
    except KeyError:
        mat_errors.append(mat_path)
        return

    for i in tqdm(range(num_trials)):
        mat_trial_index = np.argwhere(m["data"]["trial"].ravel() == (i + 1))
        # Trial not found in JAABA data
        if mat_trial_index.size == 0:
            continue
        
        try:
            mat_trial_index = mat_trial_index.item()
        except ValueError:
            mat_errors.append(mat_path)
            return

        behaviors = sorted([b.split('_')[0] for b in m['data'].dtype.names if 'scores' in b])
        
        if len(behaviors) != 6:
            continue

        all_behaviors = [
            "Lift",
            "Handopen",
            "Grab",
            "Sup",
            "Atmouth",
            "Chew"
        ]

        sorted_behaviors = [b for b in all_behaviors if b in behaviors]

        ethograms = []

        for b in sorted_behaviors:
            behavior_index = m['data'].dtype.names.index(f'{b}_labl_label')
            row = m['data'][mat_trial_index][0][behavior_index]
            row[row == -1] = 0
            ethograms.append(row)

        sorted_behaviors = [b.lower() for b in sorted_behaviors]

        ethogram = np.hstack(ethograms).T

        if check_all_behaviors(ethogram):
            hand_labels[m["data"]["exp"][i][0][0]] = ethogram
        
    return hand_labels

In [8]:
for mat in tqdm(mat_paths):
    hand_labels = get_hand_labels(str(mat))
    if hand_labels:
        df.loc[len(df.index)] = [mat.stem, hand_labels] 

100%|█████████████████████████████████████| 85/85 [00:00<00:00, 5749.61it/s]

100%|█████████████████████████████████████| 80/80 [00:00<00:00, 5993.58it/s]

100%|█████████████████████████████████████| 80/80 [00:00<00:00, 5895.02it/s]

100%|█████████████████████████████████████| 80/80 [00:00<00:00, 5989.30it/s]

100%|█████████████████████████████████████| 80/80 [00:00<00:00, 6010.32it/s]

100%|█████████████████████████████████████| 80/80 [00:00<00:00, 6032.91it/s]

100%|█████████████████████████████████████| 80/80 [00:00<00:00, 5976.71it/s]

100%|█████████████████████████████████████| 80/80 [00:00<00:00, 5988.23it/s]

100%|█████████████████████████████████████| 80/80 [00:00<00:00, 5935.06it/s]

100%|█████████████████████████████████████| 80/80 [00:00<00:00, 5933.90it/s]

100%|█████████████████████████████████████| 80/80 [00:00<00:00, 5982.78it/s]

100%|█████████████████████████████████████| 80/80 [00:00<00:00, 5983.85it/s]

100%|█████████████████████████████████████| 80/80 [00:00<00:00, 

In [9]:
df

,mat_path,full_hand_labels
0,/data/caitlin/mat_files/M234_20170328.mat,"{'M234_20170328_v004': [[0, 0, 0, 0, 0, 0, 0, ..."
1,/data/caitlin/mat_files/M234_20170331.mat,"{'M234_20170331_v001': [[0, 0, 0, 0, 0, 0, 0, ..."
2,/data/caitlin/mat_files/M234_20170403.mat,"{'M234_20170403_v001': [[0, 0, 0, 0, 0, 0, 0, ..."
3,/data/caitlin/mat_files/M234_20170412.mat,"{'M234_20170412_v001': [[0, 0, 0, 0, 0, 0, 0, ..."
4,/data/caitlin/mat_files/M234_20170414.mat,"{'M234_20170414_v002': [[0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
84,/data/caitlin/mat_files/M324_20201015.mat,"{'M324_20201015_v126': [[0, 0, 0, 0, 0, 0, 0, ..."
85,/data/caitlin/mat_files/M326_20201110.mat,"{'M326_20201110_v001': [[0, 0, 0, 0, 0, 0, 0, ..."
86,/data/caitlin/mat_files/M328_20201130_2500.mat,"{'M328_20201130_v001': [[0, 0, 0, 0, 0, 0, 0, ..."
87,/data/caitlin/mat_files/M336_20210612.mat,"{'M336_20210612_v001': [[0, 0, 0, 0, 0, 0, 0, ..."


In [10]:
mat_errors

['/data/caitlin/mat_files/M236_20170817.mat',
 '/data/caitlin/mat_files/M236_20170818.mat',
 '/data/caitlin/mat_files/M238_20170717.mat',
 '/data/caitlin/mat_files/M238_20170719.mat',
 '/data/caitlin/mat_files/M238_20170724.mat',
 '/data/caitlin/mat_files/M238_20170725.mat',
 '/data/caitlin/mat_files/M238_20170726.mat',
 '/data/caitlin/mat_files/M238_20170727.mat',
 '/data/caitlin/mat_files/M239_20170905.mat',
 '/data/caitlin/mat_files/M240_20170724FinalChecked.mat',
 '/data/caitlin/mat_files/M240_20170725FinalChecked.mat',
 '/data/caitlin/mat_files/M240_20170727FinalChecked.mat',
 '/data/caitlin/mat_files/M240_20170728FinalChecked.mat',
 '/data/caitlin/mat_files/M240_20170801FinalChecked.mat',
 '/data/caitlin/mat_files/M240_20170802FinalChecked.mat',
 '/data/caitlin/mat_files/M240_20170807FinalChecked.mat',
 '/data/caitlin/mat_files/M240_20170808FinalChecked.mat',
 '/data/caitlin/mat_files/M256_20180124_3rdCx.mat',
 '/data/caitlin/mat_files/M256_20180124_3rdCx_ORIG.mat',
 '/data/caitl

In [11]:
count = 0
for row in df.iterrows():
    count += len(row[1]["full_hand_labels"].keys())
count

135

In [12]:
df.to_hdf(path_or_buf="/data/caitlin/hand_labels.hdf", key="df")

/tmp/ipykernel_3243986/3406915199.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['mat_path', 'full_hand_labels'], dtype='object')]

  df.to_hdf(path_or_buf="/data/caitlin/hand_labels.hdf", key="df")


In [13]:
df = pd.read_hdf("/data/caitlin/hand_labels.hdf")

In [14]:
mat_errors2 = list()
def get_hand_labels(mat_path): 
    hand_labels = dict()

    m = loadmat(mat_path)
    try:
        num_trials = m["data1"]["trial"].shape[0]
        
    except KeyError:
        mat_errors2.append(mat_path)
        return

    for i in tqdm(range(num_trials)):
        mat_trial_index = np.argwhere(m["data1"]["trial"].ravel() == (i + 1))
        # Trial not found in JAABA data
        if mat_trial_index.size == 0:
            continue
        
        try:
            mat_trial_index = mat_trial_index.item()
        except ValueError:
            mat_errors2.append(mat_path)
            return

        behaviors = sorted([b.split('_')[0] for b in m['data1'].dtype.names if 'scores' in b])
        
        if len(behaviors) != 6:
            continue

        all_behaviors = [
            "Lift",
            "Handopen",
            "Grab",
            "Sup",
            "Atmouth",
            "Chew"
        ]

        sorted_behaviors = [b for b in all_behaviors if b in behaviors]

        ethograms = []

        for b in sorted_behaviors:
            behavior_index = m['data1'].dtype.names.index(f'{b}_labl_label')
            row = m['data1'][mat_trial_index][0][behavior_index]
            row[row == -1] = 0
            ethograms.append(row)

        sorted_behaviors = [b.lower() for b in sorted_behaviors]

        ethogram = np.hstack(ethograms).T

        if check_all_behaviors(ethogram):
            hand_labels[m["data1"]["exp"][i][0][0]] = ethogram

    return hand_labels

In [15]:
for mat in tqdm(mat_errors):
    hand_labels = get_hand_labels(Path(mat))
    if hand_labels:
        df.loc[len(df.index)] = [Path(mat).stem, hand_labels] 

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 5893.15it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 81/81 [00:00<00:00, 5987.96it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 5941.57it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 5940.94it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 5981.40it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 6002.47it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 6021.11it/s]


In [16]:
mat_errors2

[PosixPath('/data/caitlin/mat_files/M239_20170905.mat'),
 PosixPath('/data/caitlin/mat_files/M256_20180124_3rdCx.mat'),
 PosixPath('/data/caitlin/mat_files/M256_20180124_3rdCx_ORIG.mat'),
 PosixPath('/data/caitlin/mat_files/M258_20180430to20180502_CNO.mat'),
 PosixPath('/data/caitlin/mat_files/M258_20180502to20180504_PSEM.mat'),
 PosixPath('/data/caitlin/mat_files/M258_20180507to20180509_PSEM.mat'),
 PosixPath('/data/caitlin/mat_files/M258_20180509to20180511_CNO.mat'),
 PosixPath('/data/caitlin/mat_files/M258_20180514to20180516_PSEM.mat'),
 PosixPath('/data/caitlin/mat_files/M258_20180516to20180518_CNO.mat'),
 PosixPath('/data/caitlin/mat_files/M260_20180404_06_1stPSEM1X.mat'),
 PosixPath('/data/caitlin/mat_files/M260_20180411_13_1stPSEM3X.mat'),
 PosixPath('/data/caitlin/mat_files/M260_20180424_27_2ndPSEM3X.mat'),
 PosixPath('/data/caitlin/mat_files/M260_20180501_03_1stCNO.mat'),
 PosixPath('/data/caitlin/mat_files/M260_20180503_05_3rdPSEM3X.mat'),
 PosixPath('/data/caitlin/mat_files/

In [17]:
df

,mat_path,full_hand_labels
0,M234_20170328,"{'M234_20170328_v004': [[0, 0, 0, 0, 0, 0, 0, ..."
1,M234_20170331,"{'M234_20170331_v001': [[0, 0, 0, 0, 0, 0, 0, ..."
2,M234_20170403,"{'M234_20170403_v001': [[0, 0, 0, 0, 0, 0, 0, ..."
3,M234_20170412,"{'M234_20170412_v001': [[0, 0, 0, 0, 0, 0, 0, ..."
4,M234_20170414,"{'M234_20170414_v002': [[0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
85,M326_20201110,"{'M326_20201110_v001': [[0, 0, 0, 0, 0, 0, 0, ..."
86,M328_20201130_2500,"{'M328_20201130_v001': [[0, 0, 0, 0, 0, 0, 0, ..."
87,M336_20210612,"{'M336_20210612_v001': [[0, 0, 0, 0, 0, 0, 0, ..."
88,M336_20210613,"{'M336_20210613_v001': [[0, 0, 0, 0, 0, 0, 0, ..."


In [18]:
df.to_hdf(path_or_buf="/data/caitlin/hand_labels.hdf", key="df")

/tmp/ipykernel_3243986/3406915199.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['mat_path', 'full_hand_labels'], dtype='object')]

  df.to_hdf(path_or_buf="/data/caitlin/hand_labels.hdf", key="df")


In [19]:
df = pd.read_hdf("/data/caitlin/hand_labels.hdf")

In [20]:
count = 0
for row in df.iterrows():
    count += len(row[1]["full_hand_labels"].keys())
count

136

In [31]:
mat_errors3 = list()

def get_hand_labels(mat_path): 
    hand_labels = dict()
    m = loadmat(mat_errors2[0])

    num_trials = m["data"]["trial"].shape[0]

    for i in tqdm(range(num_trials)):
        mat_trial_index = np.argwhere(m["data"]["trial"].ravel() == (i + 1))
        # Trial not found in JAABA data
        if mat_trial_index.size == 0:
            continue

        try:
            mat_trial_index = mat_trial_index.item()
        except ValueError:
            continue

        behaviors = sorted([b.split('_')[0] for b in m['data'].dtype.names if 'scores' in b])

        if len(behaviors) != 6:
            continue

        all_behaviors = [
            "Lift",
            "Handopen",
            "Grab",
            "Sup",
            "Atmouth",
            "Chew"
        ]

        sorted_behaviors = [b for b in all_behaviors if b in behaviors]

        ethograms = []

        for b in sorted_behaviors:
            behavior_index = m['data'].dtype.names.index(f'{b}_labl_label')
            row = m['data'][mat_trial_index][0][behavior_index]
            row[row == -1] = 0
            ethograms.append(row)

        sorted_behaviors = [b.lower() for b in sorted_behaviors]

        ethogram = np.hstack(ethograms).T

        if check_all_behaviors(ethogram):
            hand_labels[m["data"]["exp"][i][0][0]] = ethogram

    return hand_labels

In [32]:
for mat in tqdm(mat_errors2):
    hand_labels = get_hand_labels(Path(mat))
    if hand_labels:
        df.loc[len(df.index)] = [Path(mat).stem, hand_labels] 

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 26/26 [00:03<00:00,  7.02it/s]


In [33]:
df

,mat_path,full_hand_labels
0,M234_20170328,"{'M234_20170328_v004': [[0, 0, 0, 0, 0, 0, 0, ..."
1,M234_20170331,"{'M234_20170331_v001': [[0, 0, 0, 0, 0, 0, 0, ..."
2,M234_20170403,"{'M234_20170403_v001': [[0, 0, 0, 0, 0, 0, 0, ..."
3,M234_20170412,"{'M234_20170412_v001': [[0, 0, 0, 0, 0, 0, 0, ..."
4,M234_20170414,"{'M234_20170414_v002': [[0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
85,M326_20201110,"{'M326_20201110_v001': [[0, 0, 0, 0, 0, 0, 0, ..."
86,M328_20201130_2500,"{'M328_20201130_v001': [[0, 0, 0, 0, 0, 0, 0, ..."
87,M336_20210612,"{'M336_20210612_v001': [[0, 0, 0, 0, 0, 0, 0, ..."
88,M336_20210613,"{'M336_20210613_v001': [[0, 0, 0, 0, 0, 0, 0, ..."


In [34]:
mat_errors3

[]

In [35]:
df.to_hdf(path_or_buf="/data/caitlin/hand_labels.hdf", key="df")

/tmp/ipykernel_3243986/3406915199.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['mat_path', 'full_hand_labels'], dtype='object')]

  df.to_hdf(path_or_buf="/data/caitlin/hand_labels.hdf", key="df")


In [36]:
df = pd.read_hdf("/data/caitlin/hand_labels.hdf")

In [37]:
count = 0
for row in df.iterrows():
    count += len(row[1]["full_hand_labels"].keys())
count

136